<a href="https://colab.research.google.com/github/lidar532/RINEX_Tools/blob/main/RINEX_ToolBox_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RINEX ToolBox Notebook for Google Colab**
Version 2021-0911-0117

C. W. Wright lidar532(at)gmail.com



## Description
A collection of tools for working with RINEX data files. This notebook provides a number of 
python wrappers around the [UNAVCO Teqc GNSS program](https://www.unavco.org/software/data-processing/teqc/teqc.html). Teqc is a powerful commandline program with huge number of 'switches'.  This notebook 
installs a Linux version of Teqc on the Colab server and then provides cells which call Teqc with 
the switches preset for various common operations.  The notbook provides the following:
* Ability to optionally read your data files from your Google Drive.
* Extract Event marks from a RINEX file and convert them to a file with data columns for GPS_Date_Time and
  for UTC_Date_Time
* Decimate a base station file to 30 second interval file suitable for sending to [NOAA NGS Online       Positioning User Service (OPUS).](https://geodesy.noaa.gov/OPUS/)
* Extract and printout the header portion of a RINEX File.
* Run a Teqc quality control analysis, and generate a report.
* Verify a RINEX file.
* Extract start and stop time from a Teqc report




# Get Started.

In [ ]:
#@title Step 2. (Optional) Mount your Google Drive { form-width: "30%", display-mode: "form" }
#@markdown If your data is on your Google Drive, running this cell will mount the drive.
#@markdown Alternatively you can just upload your RINEX file(s) using the file panel on the 
#@markdown left.
if not 'google.colab' in str(get_ipython()):
  print('You are not running is Google Colab.')
else:
  from google.colab import drive
  !rm -rf /content/sample_data/
  drive.mount('/content/.gdrive')
  !ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
  print('Your Gdrive is mounted')

In [ ]:
#@title Step 1. Teqc: Get and install teqc from UNAVCO. { form-width: "30%", display-mode: "form" }
import os
import datetime

if not os.path.isfile('/usr/local/bin/teqc'):
  print('Download and install Teqc from Unavco.')
  fpath = 'https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64d.zip'
  fn = os.path.basename(fpath)
  !wget {fpath}
  !unzip {fn}
  !mv teqc /usr/local/bin
  !rm -rf {fn}
  print('Teqc installed in /usr/local/bin.')
else:
  print('Teqc already installed.')

print(f'{datetime.datetime.utcnow()}: Operation Completed.')

#=================================================================================
def get_rinex_version(fn):
  '''
  Extract and return the version number for a RINEX file as a float.
  '''
  with open(fn) as fp:
    line = fp.readline()
  rinex_ver = float(line.split()[0])
  return rinex_ver

#=================================================================================
def get_leap_seconds(fn):
  '''
  Extract and return the GPS Leap Seconds from a RINEX file as a int.
  '''
  line_no = 0
  with open(fn) as fp:
    while True:
      line = fp.readline()
      line_no = line_no + 1
      if 'LEAP SECONDS' in line:
        leap_sec = int(line.split()[0])
        break
      if line_no > 200:
        print('NP Leap Second value found.')
        leap_sec = 0
        break
    return leap_sec



#@title **Extract event marks from RINEX** { form-width: "30%", display-mode: "form" }
#@markdown Extracts events from a RINEX file, and converts the date/time 
#@markdown to UTC.  It writes out a file with GPS_Date_Time, UTC_Date_Time
#@markdown columns.  The output file name is the same as the input with the ending reset
#@markdown to 'events.txt'

import datetime
import dateutil
import numpy as np
import os

def RINEX_event_time(dt):
  '''
  Convert an event time extracted by teqc to a numpy datetime64 and return.
  '''
  ts = dateutil.parser.parse(dt)
  npt = np.datetime64(ts)
  return npt

def events():
  if 'fnw' in globals():
    RINEX_File_Name =  fnw.value
  else:
    print('You need to run Step 3 first.')

  if not 'get_rinex_version' in globals():
    print('You must run cell 1 above first.')
  else:
    rinex_version=get_rinex_version(RINEX_File_Name)
    if  rinex_version > 2.11:
      print(f'{RINEX_File_Name} is RINEX version: {rinex_version}.   This program needs RINEX 2.11 or lower.')
    else:
      ofn = f'{os.path.splitext(RINEX_File_Name)[0]}-events.txt'
      print(f'Extracting events from RINEX file:{RINEX_File_Name}')
      print(f'Writing events to:{ofn}')
      leap_seconds = get_leap_seconds(RINEX_File_Name)
      if leap_seconds == 0:
        print('No leap second record found in your RINEX.  Enter a value:', end='')
        leap_seconds = input()

      print(f'GPS is {leap_seconds} seconds ahead of UTC.')

      ! teqc +events /tmp/events.txt  {RINEX_File_Name}  >/dev/null 2>/dev/null

      events = 0
      with open('/tmp/events.txt', 'r') as fin:
        lines = fin.readlines()
        for line in lines:
          events = events + 1
          if events == 1:
            odf = open(ofn,'w')
            print('GPS_Date_Time              UTC_Date_Time', file=odf)
          gps_dt = RINEX_event_time(line)
          utc_dt = gps_dt - np.timedelta64(leap_seconds, 's')
          print(gps_dt, utc_dt, file=odf)

        print(f'{events} events found.')
        if events >= 1:
          odf.close()
        else:
          print('No events file saved.')
        print(f'{datetime.datetime.utcnow()}:  Operation Completed.')
#@title **`teqc -O.dec 30`** Decimate RINEX to 30 second interval for OPUS { form-width: "30%", display-mode: "form" }
#@markdown Resample the RINEX file to 30 second interval for sending to the NOAA NGS [Online Positioning User Service (OPUS)](https://www.ngs.noaa.gov/OPUS/index.jsp)

# ofile = '/content/rinex-30seconds.txt'

import os

def opus():
  if 'fnw' in globals():
    RINEX_File_Name =  fnw.value
  else:
    print('You need to run Step 3 first.')

  if os.path.isfile('/usr/local/bin/teqc'):
    RINEX_File_Name
    if os.path.exists(RINEX_File_Name):
      print(f'Processing: {RINEX_File_Name}')
      rinex_version=get_rinex_version(RINEX_File_Name)
      if  rinex_version <= 2.11:
        ofile = f'{os.path.splitext(RINEX_File_Name)[0]}_OPUS.obs'
        #ofile = '/tmp/junk.txt'
        print(f'Output file:{ofile}')
        try:
          !teqc -O.dec 30 {RINEX_File_Name} 2>/dev/null > {ofile}
        except:
          print(f'{RINEX_File_Name} variable is not set.  Run cell 3. above to set the filename.')
        print(f'{RINEX_File_Name} decimated to 30 seconds in {ofile}')
        print(f'In the left panel, select "Files", "Refresh", then Double click {ofile} to view.')
      else:
        print(f'File is version: {rinex_version}.  Teqc needs version 2.11 or lower')
    else:
      print(f'{RINEX_File_Name} does not exist.')
  else:
    print('You need to run cell 1. "Teqc: Get and install teqc"')  

  print(f'{datetime.datetime.utcnow()}: Operation Completed.')
#@title **`Printout RINEX Header  `** Show the header of the RINEX file. { form-width: "30%", display-mode: "form" }
import os
def header():
  if 'fnw' in globals():
    RINEX_File_Name =  fnw.value
  else:
    print('You need to run Step 3 first.')

  if os.path.isfile('/usr/local/bin/teqc'):
    try:
      RINEX_File_Name
      if os.path.exists(RINEX_File_Name):
        with open(RINEX_File_Name) as f:
          lines = f.readlines()
          for ln in lines:
            print(ln, end='')
            if 'END OF HEADER' in ln:
              break
      else:
        print(f'{RINEX_File_Name} does not exist.')
    except:
      print(f'{RINEX_File_Name} variable is not set.  Run the cell above to set the filename.')
  else:
    print('You need to run cell 1. "Teqc: Get and install teqc"') 

  print(f'{datetime.datetime.utcnow()}: Operation Completed.')
#@title **`teqc +qc +l`** Extract start and stop time from report { form-width: "30%", display-mode: "form" }
import os

def start_stop():
  if 'fnw' in globals():
    RINEX_File_Name =  fnw.value
  else:
    print('You need to run Step 3 first.')
  if os.path.isfile('/usr/local/bin/teqc'):
    try:
      RINEX_File_Name
      if os.path.exists(RINEX_File_Name):
        print(f'Processing {RINEX_File_Name}')
        rinex_version=get_rinex_version(RINEX_File_Name)
        if  rinex_version <= 2.11:
          !teqc +qc +l {RINEX_File_Name} 2>/dev/null | grep -e'Time of' -e '4-character'
        else:
          print(f'File is version: {rinex_version}.  Teqc needs version 2.11 or lower')
      else:
        print(f'{RINEX_File_Name} does not exist.')
    except:
      print(f'{RINEX_File_Name} variable is not set.  Run the cell above to set the filename.')
  else:
    print('You need to run cell 1. "Teqc: Get and install teqc"')  
#@title **teqc +qc +l** Generate a teqc report. { form-width: "30%", display-mode: "form" }
import os
def report():
  print('report()')
  if 'fnw' in globals():
    RINEX_File_Name =  fnw.value
  else:
    print('You need to run Step 3 first.')

  if os.path.isfile('/usr/local/bin/teqc'):
    try:
      RINEX_File_Name
      if os.path.exists(RINEX_File_Name):
        print(f'Processing {RINEX_File_Name}')
        rinex_version=get_rinex_version(RINEX_File_Name)
        if  rinex_version <= 2.11:
          ! teqc +qc +l {RINEX_File_Name} 2>/dev/null
        else:
          print(f'File is version: {rinex_version}.  Teqc needs version 2.11 or lower')
      else:
        print(f'{RINEX_File_Name} does not exist.')
    except:
      print(f'{RINEX_File_Name} variable is not set.  Run the cell above to set the filename.')
  else:
    print('You need to run cell 1. "Teqc: Get and install teqc"')  

  print(f'{datetime.datetime.utcnow()}: Operation Completed.')
#@title Step 3. Select a RINEX File to use. { form-width: "30%", display-mode: "form" }
#@markdown Run this cell, and then copy a RINEX file from the gps_data sample sets and paste
#@markdown a RINEX file below.
import ipywidgets as ipw
import os
import datetime
if 'google.colab' in str(get_ipython()):
  import google.colab


clicks = 0
g_c = 0
HEADER    = 'Show Header'
REPORT    = 'Generate Report'
OPUS      = 'Prep for OPUS'
CLEAR     = 'Clear Output'
EVENTS    = 'Extract Events'
STARTSTOP = 'Start Stop Times'

def run_clicked(c):
  global clicks, g_c, out
  global cmd_buttons_w
  g_c = c
  clicks = clicks +1
  print(f'clicked: {c}')
  cmd = cmd_buttons_w.value
  print(f'cmd:{cmd} cmd_buttons:{cmd_buttons.value}')
  if cmd == HEADER:
    google.colab.output.clear()
    display( gui )
    header()
  elif cmd == REPORT:
    google.colab.output.clear()
    display( gui )
    report()
  elif cmd == OPUS:
    google.colab.output.clear()
    display( gui )
    opus()
  elif cmd == EVENTS:
    google.colab.output.clear()
    display( gui )
    events()
  elif cmd == STARTSTOP:
    google.colab.output.clear()
    display( gui )
    start_stop()
  elif cmd == CLEAR:
    google.colab.output.clear()
    display( gui )


#if 'google.colab' in str(get_ipython()):
#  RINEX_File_Name =  '/content/Gdrive/Missions/2021-0504-Sheerness-Supawna/2021-0504-Sheerness-Supawna_GNSS/bases/USGS_supbase/06411240.21O' #@param {type:"string"}

if not 'RINEX_File_Name' in globals():
  RINEX_File_Name = 'Paste a RINEX file name here.'

out = ipw.Output()

fnw = ipw.Text( description="RINEX File Name:", value=RINEX_File_Name )
fnw.layout.width = '900px'
fnw.style.description_width = '120px'
fnw.value = RINEX_File_Name

cmd_buttons_w = ipw.ToggleButtons(
    description = 'Commands:',
    options=[HEADER, REPORT, OPUS, EVENTS,  STARTSTOP, CLEAR ],
    tooltips=[
              'Show the RINEX Header',
              'Generate a Teqc report',
              'Subsample the RINEX file to 30 seconds, and generate a new file.', 
              'Extract events from the RINEX and write them to a file. The file\n '+
              'will have two columns: GPS_Date_Time and UTC_Date_Time',
              'Show the start & stop times of the RINEX file.', 'Clear the output area.'
              ]
)
run_button_w = ipw.Button(description = 'Run',
                          tooltip='Execute the selected operation.'
)
run_button_w.on_click(run_clicked)

gui = ipw.VBox( [fnw, cmd_buttons_w, run_button_w])
display( gui )

if os.path.isfile('/usr/local/bin/teqc'):
  if os.path.exists(RINEX_File_Name):
    print(f'RINEX_File_Name = {RINEX_File_Name}')
  else:
    print(f"Can't find: \'{RINEX_File_Name}\'")
else:
  print('You need to run cell 1. "Teqc: Get and install teqc"')

print(f'{datetime.datetime.utcnow()}: Operation Completed.')

In [ ]:
cmd_table = [
      { 'cmd':'Show Header',      'func': header      },
      { 'cmd':'Generate Report',  'func': report      },
      { 'cmd':'Prep for OPUS',    'func': opus        },
      { 'cmd':'Extract Events',   'func': events      },
      { 'cmd':'Start Stop Times', 'func': start_stop  }
      ]

cmd_table


# RINEX Processing and Analysis

In [ ]:
#@title **`teqc -v File  `** Verify a RINEX file format { form-width: "30%", display-mode: "form" }
import os
if 'fnw' in globals():
  RINEX_File_Name =  fnw.value
else:
  print('You need to run Step 3 first.')
  
if os.path.isfile('/usr/local/bin/teqc'):
  try:
    RINEX_File_Name
    if os.path.exists(RINEX_File_Name):
      print(f'Processing {RINEX_File_Name}')
      !teqc +v {RINEX_File_Name}
    else:
      print(f'{RINEX_File_Name} does not exist.')
  except:
    print(f'{RINEX_File_Name} variable is not set.  Run the cell above to set the filename.')
else:
  print('You need to run cell 1. "Teqc: Get and install teqc"') 

print(f'{datetime.datetime.utcnow()}: Operation Completed.')

In [ ]:
#@title **`teqc ++config`**  Dump the default teqc configuration parameters { form-width: "30%", display-mode: "form" }
if os.path.isfile('/usr/local/bin/teqc'):
  !teqc ++config
else:
  print('You need to run cell 1. "Teqc: Get and install teqc"') 

print(f'{datetime.datetime.utcnow()}: Operation Completed.')

In [ ]:
#@title **`teqc ++sym`**  Show the meaning for symbols in the teqc report. { form-width: "30%", display-mode: "form" }
if os.path.isfile('/usr/local/bin/teqc'):
  !teqc ++sym
else:
  print('You need to run cell 1. "Teqc: Get and install teqc"') 

print(f'{datetime.datetime.utcnow()}: Operation Completed.')

In [ ]:
#@title **teqc -help** Run teqc -help and show all options. { form-width: "30%", display-mode: "form" }
if os.path.isfile('/usr/local/bin/teqc'):
  !teqc -help
else:
  print('You need to run cell 1. "Teqc: Get and install teqc"')

print(f'{datetime.datetime.utcnow()}: Operation Completed.')